In [ ]:
import pandas as pd

df = pd.read_csv(r'C:/Users/risha/OneDrive/Documents/Copy of address_50.csv',encoding = 'latin')
df = df.dropna()
df = df.reset_index()

df.head()

In [ ]:
X = df.loc[df['LABEL']==1]
X = X.drop('LABEL',axis=1)
X = X['ADDRESS']
print(X.shape)

y = df['LABEL']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
text = df.copy()

corpus = []
for i in range (len(X)):
    a = re.sub('[^a-zA-Z]',' ', X[i]) 
    a = a.lower()
    a = a.split()
    
    a = ' '.join(a)
    corpus.append(a)
print(corpus)

In [ ]:
import spacy

nlp = spacy.load('en_core_web_md')

doc = []
for sent in corpus:
    doc.append(nlp(sent))


In [ ]:
print(doc[3].ents)

for i in doc[3].ents:
    print(i.text,i.label_)

In [ ]:
from spacy import displacy

displacy.render(doc, style = 'ent')

In [ ]:
print(X[0:20])

In [ ]:
import logging
import argparse
import sys
import os
import json
import pickle
training_data = []
lines = []

with open('Desktop/datamatics_internship/spacy_ner(1).json', 'r') as f:
    lines = f.readlines()
    
    for line in lines:
        data = json.loads(line)
        text = data['content']
        entities = []
        for annotation in data['annotation']:
            point = annotation['points'][0]
            labels = annotation['label']
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                entities.append((point['start'], point['end'] + 1 ,label))


        training_data.append((text, {"entities" : entities}))
    with open('training_data', 'wb') as fp:
        pickle.dump(training_data, fp)

In [ ]:
import random
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example

with open ('training_data', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

LABEL = ['AREA','CITY','STATE','LOCATION']
model = None

nlp = spacy.blank('en')  # create blank Language class
print("Created blank 'en' model")

if 'ner' not in nlp.pipe_names:
    nlp.add_pipe('ner')

ner = nlp.get_pipe('ner')

for i in LABEL:
    ner.add_label(i)

optimizer = nlp.initialize()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(10):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            for texts,annotations in batch:
                
                doc = nlp.make_doc(texts)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)

In [ ]:
test_text = '104, Plot No. 170, Jawahar Nagar Rd, Kakaji Nagar, No. 2, Goregaon West, Mumbai, Maharashtra 400062'
doc = nlp(test_text)

for ent in doc.ents:
    print(ent.label_, ent.text)